In [1]:


# Notebook whic
# 1. Downloads weights
# 2. Initializes model and imports weights
# 3. Performs test time evaluation of videos (already preprocessed with ConvertDICOMToAVI.ipynb)

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
import torch
import torchvision

sys.path.append("..")
import echonet

import wget 

#destinationFolder = "/Users/davidouyang/Dropbox/Echo Research/CodeBase/Output"
destinationFolder = "/home/lalith/echonet/dynamic/Output_test"
#videosFolder = "/Users/davidouyang/Dropbox/Echo Research/CodeBase/a4c-video-dir"
videosFolder = "/mnt/storage/Anonymized_echo_AP4"
#DestinationForWeights = "/Users/davidouyang/Dropbox/Echo Research/CodeBase/EchoNetDynamic-Weights"
DestinationForWeights = "/home/lalith/echonet/dynamic/output/segmentation/"

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

In [2]:
# Download model weights

if os.path.exists(DestinationForWeights):
    print("The weights are at", DestinationForWeights)
else:
    print("Creating folder at ", DestinationForWeights, " to store weights")
    os.mkdir(DestinationForWeights)
    
segmentationWeightsURL = 'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/deeplabv3_resnet50_random.pt'
ejectionFractionWeightsURL = 'https://github.com/douyang/EchoNetDynamic/releases/download/v1.0.0/r2plus1d_18_32_2_pretrained.pt'


if not os.path.exists(os.path.join(DestinationForWeights, os.path.basename(segmentationWeightsURL))):
    print("Downloading Segmentation Weights, ", segmentationWeightsURL," to ",os.path.join(DestinationForWeights,os.path.basename(segmentationWeightsURL)))
    filename = wget.download(segmentationWeightsURL, out = DestinationForWeights)
else:
    print("Segmentation Weights already present")
    
if not os.path.exists(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL))):
    print("Downloading EF Weights, ", ejectionFractionWeightsURL," to ",os.path.join(DestinationForWeights,os.path.basename(ejectionFractionWeightsURL)))
    filename = wget.download(ejectionFractionWeightsURL, out = DestinationForWeights)
else:
    print("EF Weights already present")
        


Creating folder at  /home/lalith/echonet/dynamic/EchoNetDynamic-Weights  to store weights


In [24]:
# Initialize and Run EF model

frames = 32
period = 1 #2
batch_size = 20
model = torchvision.models.video.r2plus1d_18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)



print("loading weights from ", os.path.join(DestinationForWeights, "r2plus1d_18_32_2_pretrained"))

if torch.cuda.is_available():
    print("cuda is available, original weights")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL)))
    model.load_state_dict(checkpoint['state_dict'])
else:
    print("cuda is not available, cpu weights")
    device = torch.device("cpu")
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(ejectionFractionWeightsURL)), map_location = "cpu")
    state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict_cpu)


# try some random weights: final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar
# scp ouyangd@arthur2:~/Echo-Tracing-Analysis/final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar "C:\Users\Windows\Dropbox\Echo Research\CodeBase\EchoNetDynamic-Weights"
#Weights = "final_r2+1d_model_regression_EF_sgd_skip1_32frames.pth.tar"


output = os.path.join(destinationFolder, "cedars_ef_output.csv")

ds = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder)
print(ds.split, ds.fnames)

mean, std = echonet.utils.get_mean_and_std(ds)

kwargs = {"target_type": "EF",
          "mean": mean,
          "std": std,
          "length": frames,
          "period": period,
          }

ds = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder, **kwargs)

test_dataloader = torch.utils.data.DataLoader(ds, batch_size = 1, num_workers = 5, shuffle = True, pin_memory=(device.type == "cuda"))
loss, yhat, y = echonet.utils.video.run_epoch(model, test_dataloader, "test", None, device, save_all=True, blocks=25)

with open(output, "w") as g:
    for (filename, pred) in zip(ds.fnames, yhat):
        for (i,p) in enumerate(pred):
            g.write("{},{},{:.4f}\n".format(filename, i, p))


Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 266, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times


loading weights from  /home/lalith/echonet/dynamic/EchoNetDynamic-Weights/r2plus1d_18_32_2_pretrained
cuda is available, original weights
EXTERNAL_TEST ['0X11109B2A42A4C76E.avi']


100%|██████████| 1/1 [00:00<00:00,  1.86it/s]


TypeError: run_epoch() got an unexpected keyword argument 'blocks'

In [9]:
# Initialize and Run Segmentation model
import skimage.draw
import tqdm
import sklearn
import scipy
torch.cuda.empty_cache()
batch_size = 20

videosFolder = "/home/lalith/scratch/converted_camus"

model_name = 'deeplabv3_resnet50'
model = torchvision.models.segmentation.__dict__[model_name](pretrained=False, aux_loss=False)
model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size=model.classifier[-1].kernel_size)  # change number of outputs to 1



print("loading weights from ", os.path.join(DestinationForWeights, "deeplabv3_resnet50_random"))

if torch.cuda.is_available():
    print("cuda is available, original weights")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(segmentationWeightsURL)))
    model.load_state_dict(checkpoint['state_dict'])
else:
    print("cuda is not available, cpu weights")
    device = torch.device("cpu")
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(segmentationWeightsURL)), map_location = "cpu")
    state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict_cpu)


def collate_fn(x):
    x, f = zip(*x)
    i = list(map(lambda t: t.shape[1], x))
    x = torch.as_tensor(np.swapaxes(np.concatenate(x, 1), 0, 1))
    return x, f, i
mean = np.array([31.834011, 31.95879,  32.082172])
std = np.array([48.866325, 49.137333, 49.361984])
dataloader = torch.utils.data.DataLoader(echonet.datasets.Echo(split="external_test", external_test_location = videosFolder, target_type=["Filename"], length=None, period=1, mean=mean, std=std),
                                         batch_size=10, num_workers=0, shuffle=False, pin_memory=(device.type == "cuda"), collate_fn=collate_fn)

output = destinationFolder
model.eval()

os.makedirs(os.path.join(output, "videos"), exist_ok=True)
os.makedirs(os.path.join(output, "size"), exist_ok=True)
echonet.utils.latexify()
flip = False
with torch.no_grad():
    with open(os.path.join(output, "size.csv"), "w") as g:
        g.write("Filename,Frame,Size,ComputerSmall\n")

        for (x, (filenames), length) in tqdm.tqdm(dataloader):
            # print(x, filenames, length)
            if flip:
                x = torch.flip(x,[3])
            # Run segmentation model on blocks of frames one-by-one
            # The whole concatenated video may be too long to run together
            y = np.concatenate([model(x[i:(i + batch_size), :, :, :].to(device))["out"].detach().cpu().numpy() for i in range(0, x.shape[0], batch_size)])

            start = 0
            
            x = x.numpy()
            for (i, (filename, offset)) in enumerate(zip(filenames, length)):
                # Extract one video and segmentation predictions
                video = x[start:(start + offset), ...]
                logit = y[start:(start + offset), 0, :, :]

                # Un-normalize video
                video *= std.reshape(1, 3, 1, 1)
                video += mean.reshape(1, 3, 1, 1)

                # Get frames, channels, height, and width
                f, c, h, w = video.shape  # pylint: disable=W0612
                print(w,h,c)
                assert c == 3

                # Put two copies of the video side by side
                video = np.concatenate((video, video), 3)

                # If a pixel is in the segmentation, saturate blue channel
                # Leave alone otherwise
                video[:, 0, :, w:] = np.maximum(255. * (logit > 0), video[:, 0, :, w:])  # pylint: disable=E1111

                # Add blank canvas under pair of videos
                video = np.concatenate((video, np.zeros_like(video)), 2)

                # Compute size of segmentation per frame
                size = (logit > 0).sum((1, 2))

                # Identify systole frames with peak detection
                trim_min = sorted(size)[round(len(size) ** 0.05)]
                trim_max = sorted(size)[round(len(size) ** 0.95)]
                trim_range = trim_max - trim_min
                systole = set(scipy.signal.find_peaks(-size, distance=20, prominence=(0.50 * trim_range))[0])

                # Write sizes and frames to file
                for (frame, s) in enumerate(size):
                    # g.write("{},{},{},{}\n".format(filename, frame, s, 1 if x in  else 0))
                    g.write("{},{},{},{}\n".format(filename, frame, s, 1 if frame in systole else 0))


                # Plot sizes
                fig = plt.figure(figsize=(size.shape[0] / 50 * 1.5, 3))
                plt.scatter(np.arange(size.shape[0]) / 50, size, s=1)
                ylim = plt.ylim()
                for s in systole:
                    plt.plot(np.array([s, s]) / 50, ylim, linewidth=1)
                plt.ylim(ylim)
                plt.title(os.path.splitext(filename)[0])
                plt.xlabel("Seconds")
                plt.ylabel("Size (pixels)")
                plt.tight_layout()
                plt.savefig(os.path.join(output, "size", os.path.splitext(filename)[0] + ".pdf"))
                plt.close(fig)

                # Normalize size to [0, 1]
                size -= size.min()
                size = size / size.max()
                size = 1 - size

                # Iterate the frames in this video
                for (f, s) in enumerate(size):

                    # On all frames, mark a pixel for the size of the frame
                    video[:, :, int(round(115 + 100 * s)), int(round(f / len(size) * 200 + 10))] = 255.

                    if f in systole:
                        # If frame is computer-selected systole, mark with a line
                        video[:, :, 115:224, int(round(f / len(size) * 200 + 10))] = 255.

                    def dash(start, stop, on=10, off=10):
                        buf = []
                        x = start
                        while x < stop:
                            buf.extend(range(x, x + on))
                            x += on
                            x += off
                        buf = np.array(buf)
                        buf = buf[buf < stop]
                        return buf
                    d = dash(115, 224)

                    # if f == large_index[i]:
                    #     # If frame is human-selected diastole, mark with green dashed line on all frames
                    #     video[:, :, d, int(round(f / len(size) * 200 + 10))] = np.array([0, 225, 0]).reshape((1, 3, 1))
                    # if f == small_index[i]:
                    #     # If frame is human-selected systole, mark with red dashed line on all frames
                    #     video[:, :, d, int(round(f / len(size) * 200 + 10))] = np.array([0, 0, 225]).reshape((1, 3, 1))

                    # Get pixels for a circle centered on the pixel
                    r, c = skimage.draw.disk((int(round(115 + 100 * s)), int(round(f / len(size) * 200 + 10))), 4.1)

                    # On the frame that's being shown, put a circle over the pixel
                    video[f, :, r, c] = 255.

                # Rearrange dimensions and save
                video = video.transpose(1, 0, 2, 3)
                video = video.astype(np.uint8)
                echonet.utils.savevideo(os.path.join(output, "videos", 'result_'+filename), video, 50)

                # Move to next video
                start += offset

/home/lalith/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lalith/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


loading weights from  /home/lalith/echonet/dynamic/EchoNetDynamic-Weights/deeplabv3_resnet50_random
cuda is available, original weights


  0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-9-e1a9b4307ba9>:115: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()
<ipython-input-9-e1a9b4307ba9>:115: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()
100%|██████████| 1/1 [00:00<00:00,  3.39it/s]

112 112 3
112 112 3


In [ ]:
# Code for just getting output
import pathlib
import tqdm
import scipy
torch.cuda.empty_cache()


model_name = 'deeplabv3_resnet50'
model = torchvision.models.segmentation.__dict__[model_name](pretrained=False, aux_loss=False)
model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size=model.classifier[-1].kernel_size)  # change number of outputs to 1
def smooth_segmentation(x,alpha):
    """
    This function smooths the segmentation over between frames.
    The logic is:
    the nth frame = alpha * the nth frame + (1-alpha) * the n-1th frame
    """
    final = [x[0]]
    for i in x[1:]:
        final.append(alpha*i+(1-alpha)*final[-1])
    return np.array(final)


print("loading weights from ", os.path.join(DestinationForWeights, "deeplabv3_resnet50_random"))

if torch.cuda.is_available():
    print("cuda is available, original weights")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(segmentationWeightsURL)))
    model.load_state_dict(checkpoint['state_dict'])
else:
    print("cuda is not available, cpu weights")
    device = torch.device("cpu")
    checkpoint = torch.load(os.path.join(DestinationForWeights, os.path.basename(segmentationWeightsURL)), map_location = "cpu")
    state_dict_cpu = {k[7:]: v for (k, v) in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict_cpu)

videosFolder = "/home/lalith/scratch/converted_echo_abnormal_AP4"

ds = echonet.datasets.Echo(split = "external_test", external_test_location = videosFolder)
print(ds.split, ds.fnames)

mean, std = echonet.utils.get_mean_and_std(ds,batch_size=1)

def collate_fn(x):
    x, f = zip(*x)
    i = list(map(lambda t: t.shape[1], x))
    x = torch.as_tensor(np.swapaxes(np.concatenate(x, 1), 0, 1))
    return x, f, i

test_ds = echonet.datasets.Echo(split="external_test", external_test_location = videosFolder, target_type=["Filename"], length=None, period=1, mean=mean, std=std)
dataloader = torch.utils.data.DataLoader(test_ds, batch_size=1, num_workers=8, shuffle=False, pin_memory=(device.type == "cuda"), collate_fn=collate_fn)
block = 1024
model.eval()
alpha = 0.9
flip = True
with torch.no_grad():
    for (x, f, i) in tqdm.tqdm(dataloader):
        if flip:
            x = torch.flip(x,[3])
        x = x.to(device)
        y = np.concatenate([model(x[i:(i + block), :, :, :])["out"].detach().cpu().numpy() for i in range(0, x.shape[0], block)]).astype(np.float16)
        start = 0
        oldvideo = x.cpu().numpy().copy()
        oldvideo = oldvideo * std.reshape(1, 3, 1, 1)
        oldvideo = oldvideo + mean.reshape(1, 3, 1, 1)
        newvideo = oldvideo.copy()
        output_folder = os.path.join(destinationFolder, 'videos')
        # This is the line that smooths the segmentation
        newvideo[:,2,:,:] = np.maximum(newvideo[:,2,:,:], 255. * (smooth_segmentation(y[:, 0, :, :],alpha) > 0))
        for (filename, offset) in zip(f,i):
            # This line also smooths the segmentation
            np.save(os.path.join(output_folder, os.path.splitext(filename)[0]), smooth_segmentation(y[start:(start+offset), 0, :, :],alpha))
            #plain videos
            echonet.utils.savevideo(os.path.join(output_folder,os.path.splitext(filename)[0] + ".avi"), np.transpose(newvideo[start:(start+offset), :, :, :],(1,0,2,3)).astype(np.uint8), 65)
            # shutil.rmtree('temp')
            print(os.path.join(output_folder,os.path.splitext(filename)[0] + ".avi"))

In [7]:
!export CUDA_VISIBLE_DEVICES=0 
